In [2]:
# import libraries
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import joblib
import spacy
import re
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
# run once
# !python -m spacy download en_core_web_lg

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
# load spacy model
nlp = spacy.load("en_core_web_lg")

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Baseline Nearest Neighbors Model

In [5]:
# read in data
data_path = "../data/"

def wrangle(data_path):

    # reads in the data
    df_tracks = pd.read_csv(data_path + 'tracks.csv', parse_dates=['release_date'])
    df_lyrics = pd.read_csv(data_path + 'lyrics.csv') #usecols=['SName', 'Lyric']
    
    # drop null values on df_tracks
    df_tracks.dropna(inplace=True)
    df_lyrics.dropna(inplace=True)
    
    # case normalization on song names
    df_lyrics['SName'] = df_lyrics['SName'].apply(lambda x: x.lower())
    df_tracks['name'] = df_tracks['name'].apply(lambda x: x.lower())
    
    # clean up df_lyrics data for merging
    df_lyrics = df_lyrics.rename(columns={'ALink': 'Artist', 'SName': 'name'})
    df_lyrics['Artist'] = df_lyrics['Artist'].apply(lambda x: x.rstrip('/').lstrip('/').replace('-', " ").lower())

    # drops duplicate lyrics 
    df_lyrics = df_lyrics.drop_duplicates(subset=['Artist', 'name'])
    
    # clean up df_tracks data for merging
    df_tracks = df_tracks.rename(columns={'artists': 'Artist'})

    df_tracks['Artist'] = df_tracks['Artist'].apply(lambda x: x.lstrip("['").rstrip("']").lower())
    
    # use regex to strip acoustic/remix versions
    strip_post_dash = ' - \S+'
    strip_version = ' version'

    df_tracks['name'] = df_tracks['name'].apply(lambda song_name: re.sub(strip_post_dash, "", song_name))
    df_tracks['name'] = df_tracks['name'].apply(lambda song_name: re.sub(strip_version, "", song_name))
    
    # drop duplicate rows
    df_tracks = df_tracks.drop_duplicates(subset=['Artist', 'name'])
    
    df_merged = pd.merge(df_lyrics, df_tracks, on=['Artist', 'name'], how='inner')
    
    # drop unwanted columns
    df_merged = df_merged.drop(columns=['duration_ms'])
    
    df_merged = df_merged[df_merged['Idiom'] == 'ENGLISH']
    
    # original merge dataframes
    # df_merged = pd.merge(df_tracks, df_lyrics, how='inner', left_on=['name'], right_on='SName')
    
    
    # original drop columns not used in NN model
    # df_merged = df_merged.drop(columns= ['id', 'name', 'artists', 'id_artists', 'release_date', ])
    
    return df_merged

def keep_wanted_columns(df):
    df_dropped = df[['popularity', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]
    
    #drop(columns= ['Artist', 'name', 'SLink', 'Lyric', 'Idiom', 'id_artists', 'id', 'release_date'])
    
    return df_dropped

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:31: DeprecationWarning: invalid escape sequence \S
<>:31: DeprecationWarning: invalid escape sequence \S
<ipython-input-5-3d0839ad7a08>:31: DeprecationWarning: invalid escape sequence \S
  strip_post_dash = ' - \S+'


In [6]:
df_merged = wrangle(data_path)

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
df_merged

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Artist,name,SLink,Lyric,Idiom,id,popularity,explicit,id_artists,release_date,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,12 stones,world so cold,/12-stones/world-so-cold.html,"It starts with pain, followed by hate. Fueled ...",ENGLISH,471eQ7hcJ7JdGY1NzMmUeg,55,0,['0DrXhci3WAyo0WJv1RBOG6'],2007-01-01,...,4,-4.949,0,0.0443,0.472000,0.000000,0.360,0.394,186.227,3
1,12 stones,anthem for the underdog,/12-stones/anthem-for-the-underdog.html,You say you know just who I am. But you can't ...,ENGLISH,6FFwt1ea9hJ4MfMQLywahm,59,0,['0DrXhci3WAyo0WJv1RBOG6'],2007-01-01,...,5,-3.424,1,0.0626,0.000651,0.000000,0.339,0.468,93.977,3
2,12 stones,we are one,/12-stones/we-are-one.html,We walk alone. In the unknown. We live to win ...,ENGLISH,4lhqal0Hq63U2wETCeBdG1,58,0,['0DrXhci3WAyo0WJv1RBOG6'],2010-01-01,...,5,-4.041,0,0.1140,0.000119,0.000016,0.233,0.306,127.102,4
3,3 doors down,here without you,/3-doors-down/here-without-you.html,"A hundred days have made me older,. Since the ...",ENGLISH,3NLrRZoMF0Lx6zTlYqeIo4,76,0,['2RTUTCvo6onsAnheUk3aL9'],2002-11-12,...,10,-6.817,0,0.0252,0.049200,0.000000,0.205,0.233,143.994,4
4,3 doors down,when i'm gone,/3-doors-down/when-im-gone.html,There's another world inside of me that you ma...,ENGLISH,3WbphvawbMZ8FyqDxYGdSQ,70,0,['2RTUTCvo6onsAnheUk3aL9'],2002-11-12,...,7,-5.611,1,0.0284,0.003850,0.000000,0.103,0.374,148.095,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14394,whitney houston,you'll never stand alone,/whitney-houston/youll-never-stand-alone.html,Verse 1:. If there's a time when the tears sho...,ENGLISH,3KFbjIFQxMbOKdZ4S3lv4S,34,0,['6XpaIBNiVzIetEPCWDvAFP'],1998-11-17,...,9,-6.943,1,0.0254,0.105000,0.000000,0.272,0.194,103.907,4
14395,whitney houston,you're still my man,/whitney-houston/youre-still-my-man.html,You'Re Still My Man. On the day that you left ...,ENGLISH,2csRQWI7A2FjLYBQBiAIoi,42,0,['6XpaIBNiVzIetEPCWDvAFP'],1987-06-02,...,10,-9.496,1,0.0295,0.684000,0.000004,0.226,0.308,131.743,4
14441,zendaya,replay,/zendaya/replay.html,"Make it stop, sounds so good. I just can't tak...",ENGLISH,7d1CFwrBmH34gmS0Hkbfbt,67,0,['6sCbFbEjbYepqswM1vWjjs'],2013-01-01,...,6,-6.682,0,0.1210,0.041600,0.000200,0.302,0.618,140.023,4
15306,jorge ben jor,brother,/jorge-ben-jor/brother.html,"Brother, Brother, Prepare one more happy way f...",ENGLISH,1LLRVZB4fz7bduFEZ22DE4,48,0,['5JYtpnUKxAzXfHEYpOeeit'],1974-01-01,...,2,-11.125,1,0.0378,0.416000,0.000000,0.233,0.972,132.249,4


In [11]:
# user selects what song
user_selected_song = 'imagine'
user_selected_artist = 'john lennon'

song_row = df_merged[(df_merged['name'] == user_selected_song) & (df_merged['Artist'] == user_selected_artist)]
song_row

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Artist,name,SLink,Lyric,Idiom,id,popularity,explicit,id_artists,release_date,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
3776,john lennon,imagine,/john-lennon/imagine.html,Imagine there's no heaven. It's easy if you tr...,ENGLISH,4lwh5BoHBFGjKNAF3sNNCK,25,0,['4x1nvY2FN8jxqAFA0DA02H'],1971-09-09,...,0,-11.407,1,0.029,0.947,0.00003,0.0689,0.218,74.4,4


In [7]:
# drop categorical data, prepare for input to model
song_row = keep_wanted_columns(song_row)
df_dropped = keep_wanted_columns(df_merged)

# instantiate Nearest Neighbors model
nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')

# fit the model
nn.fit(df_dropped)

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NearestNeighbors(algorithm='kd_tree')

In [8]:
neigh_dist, neigh_index = nn.kneighbors(song_row)

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
neigh_index = neigh_index[0][1:]
neigh_index

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([ 8720,  4395, 10053,  9364])

In [10]:
for index in neigh_index:
    print(df_merged['name'].iloc[index])

someone like you
tainted love
drowning
handlebars


/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
neigh_dist

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[0.        , 1.84631722, 2.55340212, 2.58756591, 3.14668559]])

In [12]:
# joblib.dump(nn, '../data/NearestNeighborModel')

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Natural Language Processing

In [13]:
# df_merged = df_merged[df_merged['Idiom'] == 'ENGLISH']

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
import re
def clean_data(text):
    """
    Accepts a single text document and performs several regex substitutions in order to clean the document.
    Parameters
    ----------
    text: string or object
    Returns
    -------
    text: string or object
    """
    special_chars_regex = '[:?,\>$|!\'"]'
    white_spaces_regex = '[ ]{2,}'
    text = re.sub('[^a-zA-Z ]', "", text)
    text = re.sub(special_chars_regex, " ", text)
    text = re.sub(white_spaces_regex, " ", text)
    return text.lower()

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:12: DeprecationWarning: invalid escape sequence \>
<>:12: DeprecationWarning: invalid escape sequence \>
<ipython-input-14-2a9d641a4093>:12: DeprecationWarning: invalid escape sequence \>
  special_chars_regex = '[:?,\>$|!\'"]'


In [15]:
def tokenize(doc):
# run the clean_data function on the text
    text = clean_data(doc)
# Instantiatie language model
    doc = nlp(text)
    STOP_WORDS = nlp.Defaults.stop_words.union('', 'the', 'this', '-pron-', 'wo', 've', 'nt')
    return [token.lemma_.lower().lstrip().rstrip() for token in doc
            if (token.text not in STOP_WORDS)
            and (token.is_punct != True)
            and (token.text not in '')
            and (len(token.text)>2)]

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager
# !pip install pandarallel

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=7)

INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
lemmas = df_merged['Lyric'].parallel_apply(tokenize)

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
lemmas

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0        [start, pain, follow, hate, fuel, endless, que...
1        [know, imagine, wait, line, think, stand, tire...
2        [walk, unknown, live, win, victory, young, die...
3        [day, old, time, see, pretty, face, thousand, ...
4        [world, inside, secret, life, hide, darkness, ...
                               ...                        
14394    [verse, time, tear, fill, eye, past, shadow, s...
14395    [man, day, leave, say, regret, bond, break, fe...
14441    [stop, sound, good, turn, turn, know, know, st...
15306    [brother, brother, prepare, happy, way, lord, ...
15331    [mama, mama, mama, easy, girl, easy, girl, jor...
Name: Lyric, Length: 12780, dtype: object

In [20]:
id2words = corpora.Dictionary(lemmas)
corpus = [id2words.doc2bow(lemma) for lemma in lemmas]

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
lda_multicore = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                                        id2word=id2words,
                                                        num_topics=5,
                                                        chunksize=100,
                                                        passes=10,
                                                        per_word_topics=True,
                                                        workers=6)

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
# somewhat unsure of what this is doing

all_topics = lda_multicore.get_document_topics(corpus[3800], per_word_topics=True)
# for doc_topics, word_topics, phi_values in all_topics:
#     print('New Document \n')
#     print('Document topics:', doc_topics)
#     print('Word topics:', word_topics)
#     print('Phi values:', phi_values)
#     print(" ")
#     print('-------------- \n')
all_topics[0]

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(4, 0.9928047)]

In [23]:
# display the print_topics from the model 
doc_topics = lda_multicore.get_document_topics(corpus, per_word_topics=True)

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
doc_topics.corpus

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 1),
  (1, 1),
  (2, 4),
  (3, 3),
  (4, 6),
  (5, 1),
  (6, 1),
  (7, 2),
  (8, 5),
  (9, 2),
  (10, 1),
  (11, 1),
  (12, 4),
  (13, 4),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 3),
  (18, 1),
  (19, 2),
  (20, 1),
  (21, 6),
  (22, 4),
  (23, 2),
  (24, 1),
  (25, 3),
  (26, 4),
  (27, 1),
  (28, 4),
  (29, 1),
  (30, 4),
  (31, 1),
  (32, 1),
  (33, 4),
  (34, 3),
  (35, 1),
  (36, 2),
  (37, 1),
  (38, 1),
  (39, 2),
  (40, 1),
  (41, 1),
  (42, 3),
  (43, 2),
  (44, 1),
  (45, 2),
  (46, 1),
  (47, 2),
  (48, 1),
  (49, 5),
  (50, 1),
  (51, 2),
  (52, 2),
  (53, 1),
  (54, 1),
  (55, 12)],
 [(17, 2),
  (22, 3),
  (28, 1),
  (36, 2),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 5),
  (65, 1),
  (66, 1),
  (67, 2),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 3),
  (73, 1),
  (74, 1),
  (75, 3),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1)],
 [(30, 4),
  (47, 1),
  (52, 3),
  (73, 3),
  (81, 2),
  (82, 1),
  (83,

In [25]:
len(lemmas[0])

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


130

In [26]:
doc_topics.corpus

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 1),
  (1, 1),
  (2, 4),
  (3, 3),
  (4, 6),
  (5, 1),
  (6, 1),
  (7, 2),
  (8, 5),
  (9, 2),
  (10, 1),
  (11, 1),
  (12, 4),
  (13, 4),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 3),
  (18, 1),
  (19, 2),
  (20, 1),
  (21, 6),
  (22, 4),
  (23, 2),
  (24, 1),
  (25, 3),
  (26, 4),
  (27, 1),
  (28, 4),
  (29, 1),
  (30, 4),
  (31, 1),
  (32, 1),
  (33, 4),
  (34, 3),
  (35, 1),
  (36, 2),
  (37, 1),
  (38, 1),
  (39, 2),
  (40, 1),
  (41, 1),
  (42, 3),
  (43, 2),
  (44, 1),
  (45, 2),
  (46, 1),
  (47, 2),
  (48, 1),
  (49, 5),
  (50, 1),
  (51, 2),
  (52, 2),
  (53, 1),
  (54, 1),
  (55, 12)],
 [(17, 2),
  (22, 3),
  (28, 1),
  (36, 2),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 5),
  (65, 1),
  (66, 1),
  (67, 2),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 3),
  (73, 1),
  (74, 1),
  (75, 3),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1)],
 [(30, 4),
  (47, 1),
  (52, 3),
  (73, 3),
  (81, 2),
  (82, 1),
  (83,

In [27]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_multicore, corpus, id2words)
vis

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.172603 -0.083119       1        1  38.356537
4      0.028375 -0.145206       2        1  24.126086
3      0.083221  0.114331       3        1  18.493122
2     -0.035568  0.175041       4        1   9.555195
1     -0.248631 -0.061047       5        1   9.469060, topic_info=     Term          Freq         Total Category  logprob  loglift
33   love  22606.000000  22606.000000  Default  30.0000  30.0000
110  yeah  10334.000000  10334.000000  Default  29.0000  29.0000
113  baby  11019.000000  11019.000000  Default  28.0000  28.0000
115   get  16114.000000  16114.000000  Default  27.0000  27.0000
236   gon   8085.000000   8085.000000  Default  26.0000  26.0000
..    ...           ...           ...      ...      ...      ...
159  body    332.904021   1411.855048   Topic5  -5.8506   0.9123
494  real    342.064894   1748.479069   Topic5  -5.8235   0.7256
752  stop    347.027232   2637.963989   Topic5  -5.8091   0.3288
70   know    439.360342  21981.024400   Topic5  -5.5732  -1.5555
145   ill    369.485976   7998.106177   Topic5  -5.7464  -0.7177

[438 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
3660       4  0.991664       aaah
7674       4  0.997804        aah
14895      3  0.981784       aahh
7020       3  0.980632       amen
8687       4  0.994011      annie
...      ...       ...        ...
111        4  0.007362  yesterday
1459       2  0.919294       york
1459       3  0.003549       york
1459       4  0.074537       york
1459       5  0.003549       york

[1083 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 4, 3, 2])